In [68]:
import cv2
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os


In [70]:
model_path = "pb"
model = tf.saved_model.load(model_path)
movenet = model.signatures['serving_default']


In [73]:
ret, frame = cap.read()
frame_height, frame_width, _ = frame.shape


In [74]:
LEFT_ELBOW_IDX = 7
RIGHT_ELBOW_IDX = 8


In [75]:
with open('punch_coordinates2.txt', 'w') as f:

    # Read the video file.
    cap = cv2.VideoCapture('video1.mp4')

    while True:
        # Read a frame from the video.
        ret, frame = cap.read()

        if not ret:
            # End of video.
            break

        # Get the frame width and height.
        frame_height, frame_width, _ = frame.shape

        # Resize and pad the image to fit the expected input shape.
        image = tf.expand_dims(frame, axis=0)
        image = tf.cast(tf.image.resize_with_pad(
            image, 192, 192), dtype=tf.int32)

        # Run model inference.
        outputs = movenet(image)

        # Get the keypoint coordinates and confidence scores.
        keypoints = np.squeeze(outputs['output_0'])

        # Get the coordinates of the left and right elbows.
        left_elbow_coords = None
        right_elbow_coords = None

        if keypoints.ndim == 2:
            # If only one person is detected in the frame.
            if LEFT_ELBOW_IDX < keypoints.shape[0]:
                left_elbow_coords = (int(keypoints[LEFT_ELBOW_IDX][1] * frame_width), int(
                    keypoints[LEFT_ELBOW_IDX][0] * frame_height))
            if RIGHT_ELBOW_IDX < keypoints.shape[0]:
                right_elbow_coords = (int(keypoints[RIGHT_ELBOW_IDX][1] * frame_width), int(
                    keypoints[RIGHT_ELBOW_IDX][0] * frame_height))

        elif keypoints.ndim == 3:
            # If multiple people are detected in the frame, use the person with the highest confidence score.
            max_score_idx = np.argmax(keypoints[:, :, 2])
            if LEFT_ELBOW_IDX < keypoints.shape[1]:
                left_elbow_coords = (int(
                    keypoints[max_score_idx, LEFT_ELBOW_IDX][1] * frame_width), int(keypoints[max_score_idx, LEFT_ELBOW_IDX][0] * frame_height))
            if RIGHT_ELBOW_IDX < keypoints.shape[1]:
                right_elbow_coords = (int(
                    keypoints[max_score_idx, RIGHT_ELBOW_IDX][1] * frame_width), int(keypoints[max_score_idx, RIGHT_ELBOW_IDX][0] * frame_height))

        # Save the coordinates of both elbows in the file.
        if left_elbow_coords is not None and right_elbow_coords is not None:
            f.write(f"{left_elbow_coords},{right_elbow_coords}\n")

    # Release the video capture object.
    cap.release()
